In [1]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import tweepy
import time
import pandas as pd
from textblob import TextBlob
from reference import consumer_key, consumer_key_secret, access_token, access_token_secret

In [2]:
auth = tweepy.OAuthHandler(consumer_key, consumer_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [3]:
search_words = "gillette OR @Gillette OR to:Gillette -stadium -filter:retweets -from:Gillette -#sponsored"
date_since = "2018-06-16"

In [14]:
# Additional paramaters per https://github.com/KTakatsuji/Twitter-Sentiment-Naive-Bayes/blob/master/NaiveBayes/part1.py

count = 0
start = 0
errorCount=0

#here we tell the program how fast to search 
waitquery = 100      #this is the number of searches it will do before resting
waittime = 2.0          # this is the length of time we tell our program to rest
total_number = 1500     #this is the total number of queries we want
justincase = 1         #this is the number of minutes to wait just in case twitter throttles us
results = []

users =tweepy.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since).items()

In [15]:
# functions that allow waiting
text = [0] * total_number
secondcount = 0
idvalues = [1] * total_number
 #1 is happy; 2 is sad; 3 is angry; 4 is fearful
#Below is where the magic happens and the queries are being made according to our desires above
while secondcount < total_number:
    try:
        user = next(users)
        count += 1
        
        #We say that after every 100 searches wait 5 seconds
        if (count%waitquery == 0):
            time.sleep(waittime)
            #break

    except tweepy.TweepError:
        #catches TweepError when rate limiting occurs, sleeps, then restarts.
        #nominally 15 minnutes, make a bit longer to avoid attention.
        print("sleeping....")
        time.sleep(60*justincase)
        user = next(users)
    except StopIteration:
        break
    
    try:
        results.append(user.text)
        secondcount = secondcount + 1
        print("current saved is:"+str(secondcount))

    except UnicodeEncodeError:
        errorCount += 1
        print("UnicodeEncodeError,errorCount ="+str(errorCount))


current saved is:1
current saved is:2
current saved is:3
current saved is:4
current saved is:5
current saved is:6
current saved is:7
current saved is:8
current saved is:9
current saved is:10
current saved is:11
current saved is:12
current saved is:13
current saved is:14
current saved is:15
current saved is:16
current saved is:17
current saved is:18
current saved is:19
current saved is:20
current saved is:21
current saved is:22
current saved is:23
current saved is:24
current saved is:25
current saved is:26
current saved is:27
current saved is:28
current saved is:29
current saved is:30
current saved is:31
current saved is:32
current saved is:33
current saved is:34
current saved is:35
current saved is:36
current saved is:37
current saved is:38
current saved is:39
current saved is:40
current saved is:41
current saved is:42
current saved is:43
current saved is:44
current saved is:45
current saved is:46
current saved is:47
current saved is:48
current saved is:49
current saved is:50
current s

current saved is:397
current saved is:398
current saved is:399
current saved is:400
current saved is:401
current saved is:402
current saved is:403
current saved is:404
current saved is:405
current saved is:406
current saved is:407
current saved is:408
current saved is:409
current saved is:410
current saved is:411
current saved is:412
current saved is:413
current saved is:414
current saved is:415
current saved is:416
current saved is:417
current saved is:418
current saved is:419
current saved is:420
current saved is:421
current saved is:422
current saved is:423
current saved is:424
current saved is:425
current saved is:426
current saved is:427
current saved is:428
current saved is:429
current saved is:430
current saved is:431
current saved is:432
current saved is:433
current saved is:434
current saved is:435
current saved is:436
current saved is:437
current saved is:438
current saved is:439
current saved is:440
current saved is:441
current saved is:442
current saved is:443
current saved

current saved is:791
current saved is:792
current saved is:793
current saved is:794
current saved is:795
current saved is:796
current saved is:797
current saved is:798
current saved is:799
current saved is:800
current saved is:801
current saved is:802
current saved is:803
current saved is:804
current saved is:805
current saved is:806
current saved is:807
current saved is:808
current saved is:809
current saved is:810
current saved is:811
current saved is:812
current saved is:813
current saved is:814
current saved is:815
current saved is:816
current saved is:817
current saved is:818
current saved is:819
current saved is:820
current saved is:821
current saved is:822
current saved is:823
current saved is:824
current saved is:825
current saved is:826
current saved is:827
current saved is:828
current saved is:829
current saved is:830
current saved is:831
current saved is:832
current saved is:833
current saved is:834
current saved is:835
current saved is:836
current saved is:837
current saved

In [ ]:
# results = [] 

# for tweet in tweepy.Cursor(api.search,
#               q=search_words,
#               lang="en",
#               since=date_since).items(10):
#     results.append(tweet.text)
    
# results

In [16]:
all_tweet_data = pd.DataFrame()

for tweet in results:
    analysis = TextBlob(tweet)
    if analysis.sentiment[0]>0:
       score = 1
    elif analysis.sentiment[0]<0:
       score = -1
    else:
       score = 0
    all_tweet_data = all_tweet_data.append({'Text': tweet, 'Score': score}, ignore_index=True)
    
all_tweet_data

,Score,Text
0,1.0,Check out our special on ProGlide Gillette Fus...
1,1.0,@evertonian89 @Catherw96287611 @schoudshady We...
2,0.0,@gavinworby Gillette Soccer Saturday
3,0.0,@Fatbob2013 @zephstefkrysezi Don’t forget abou...
4,1.0,@smh @sarah_keoghan Good to hear! Won't ever u...
5,0.0,A lot of Italy today and a bit of Germany prec...
6,-1.0,This is grotesque but Gillette have the right ...
7,0.0,@Dove @ReverieAbsinthe You're a corporation pa...
8,0.0,Laura your forgetting the protests against Hic...
9,1.0,Free Gillette Skinguard Razors for US Active M...


In [11]:
#remaining steps, switch to larger amounts of data
#sort for train and test
#export to seperate .csv files

test_data = pd.DataFrame()
train_data = pd.DataFrame()

counter = 1;

for index, i in all_tweet_data.iterrows():
    if counter % 2 == 0:
        train_data = train_data.append({'Text': i[1], 'Score': i[0]}, ignore_index=True)
    else:
        test_data = test_data.append({'Text': i[1], 'Score': i[0]}, ignore_index=True)
    counter += 1

In [12]:
#Training Data
train_data

train_data.to_csv("Data/train_data.csv")

In [13]:
#Testing Data
test_data
test_data.to_csv("Data/test_data.csv")